
# Module 2 - In this jupyter notebook, the seasonal T, ET, RET, NPP are calculated 

* Step 2a - Set up: Import modules/libraries, inport data, create output folder
* Step 2b - Defining function 
* Step 2c - Calculate seasonal T, ET, RET, ETp, NPP

**=====================================================================================================================**

![title](img/Fig2_1.png)

**=====================================================================================================================**

## Step 2a - Set up

## i) Import modules/libraries

In [1]:
import os                                 # a module for interacting with the operating system
import sys
import glob                               # used to retrieve files/pathnames matching a specified pattern
import re                                 # re sub() module can be used to replace substring

import pandas as pd                       # to store and manipulate tabular data in rows of observations and columns of variables
import numpy as np                        # stands for 'Numerical Python, is a python library used for scientific computing with arrays
import calendar
import datetime
from matplotlib import pyplot as plt      # is a plotting library used for 2D graphics in python 

# change the directory to where the modules are saved
os.chdir(os.path.join(os.path.split(os.getcwd())[0], "Modules"))
from GIS_functions import GIS_function as gis

## ii) Import the input data:
* WaPOR data (T, AETI, REF, NPP), 
* Dates corresponding to the raster layer and 
* Start and end of cropseason and crop coefficient (Kc) of crop

## * Import raster (WaPOR) data

In [2]:
dir_proj = os.path.split(os.getcwd())[0]  
dir_data = "Data"

# transpiration, evapotranspiration & interceptio and reference evapotranspiration
input_folderT = os.path.join(dir_proj, dir_data, "1_L2_T_filtered") 
input_fhsT = glob.glob(input_folderT + '\*.tif')  # glob.glob returns the list of files with their full path

input_folderET = os.path.join(dir_proj, dir_data, "1_L2_AETI_filtered") 
input_fhsET = glob.glob(input_folderET + '\*.tif')   

input_folderRET = os.path.join(dir_proj, dir_data, "1_L1_RET_filtered") 
input_fhsRET = glob.glob(input_folderRET + '\*.tif')

input_folderNPP = os.path.join(dir_proj, dir_data, "1_L2_NPP_filtered") 
input_fhsNPP = glob.glob(input_folderNPP + '\*.tif')

## ** Import the dates corresponding to the raster layers of WaPOR (cube code)

![title](img/Fig2_2.jpg), ![title](img/Fig2_3.png)

* Running the following cell activate the 'WaPOR' module that requires internet connection and takes longtime to download the the catalog 
* <span style='background :lightgreen' > **Run the cell only one time, if the latest date in the 'time_range' is updated.**
* <span style='background :pink' > While running this section for the first time, the the catalog info is saved in the working directory in excel file (e.g. df_availET.xlsx).
* <span style='background :pink' > Running the notebook for the second time should skip this cell, but run the next one (# read the df_avial from already saved excel under the Data) 

In [3]:
import os 
os.chdir(os.path.join(os.path.split(os.getcwd())[0], "Modules"))
import WaPOR                                # API to interact with WaPOR portal
WaPOR.API.version = 2

# read the cube info (dataframe) from the cataloge 
cube_codeT   = 'L2_T_D' 
cube_codeET  = 'L2_AETI_D' 
cube_codeRET = 'L1_RET_D' 
cube_codeNPP = 'L2_NPP_D' 

time_range   = '2009-01-01,2019-12-31'

df_availT    = WaPOR.API.getAvailData(cube_codeT,   time_range)
df_availET   = WaPOR.API.getAvailData(cube_codeET,  time_range)
df_availRET  = WaPOR.API.getAvailData(cube_codeRET, time_range)
df_availNPP  = WaPOR.API.getAvailData(cube_codeNPP, time_range)

# save the dataframe to excel to access it offline
output_folder = os.path.join(os.path.split(os.getcwd())[0], "Data") 

df_availT.to_excel(os.path.join(output_folder,   'df_availT.xlsx'))
df_availET.to_excel(os.path.join(output_folder,  'df_availET.xlsx'))
df_availRET.to_excel(os.path.join(output_folder, 'df_availRET.xlsx'))
df_availNPP.to_excel(os.path.join(output_folder, 'df_availNPP.xlsx'))

# Get personaL WAPOR API Token by registering in the top right cornor of the page: wapor.apps.fao.org/home/1

In [4]:
# read the df_avial from already saved excel 

time_range  = '2009-01-01,2019-12-31'

df_availT   = pd.read_excel('../data/df_availT.xlsx')
df_availET  = pd.read_excel('../data/df_availET.xlsx')
df_availRET = pd.read_excel('../data/df_availRET.xlsx')
df_availNPP = pd.read_excel('../data/df_availNPP.xlsx')

## *** Define and import the Start Of crop Season (SOS) and End Of crop Season (EOS)
* Edit the start and end of crop seasons **in the df_SOsEos.xlsx file in the data folder**
* You can add or delete rows depending on the number of seasons 

In [5]:
df_dates = pd.read_excel('../data/df_SosEos.xlsx')
df_dates

## **** Define and import the Kc per month
* Edit the **months** the corresponding **Kc** value in the order of start of crop season (inital stage) to end of crop season (late-season stage) **in the df_Kc.xlsx file in the data folder**
* The rows should be for months within the duration of the crop season 
* The figure below shows the Kc curve of a sugarcane, given as example
![title](img/Fig3_2.PNG)

In [6]:
df_kc = pd.read_excel('../data/df_Kc.xlsx')
df_kc 

## iii) Output folder: Make one or connect to the existing one

In [7]:
# the directory of the output folder
dir_proj = os.path.split(os.getcwd())[0]  
dir_data = "Data"
output_folderT           = os.path.join(dir_proj, dir_data, "2L2_T_season") 
output_folderET          = os.path.join(dir_proj, dir_data, "2L2_AETI_season") 
output_folderRET         = os.path.join(dir_proj, dir_data, "2L1_RET_season") 
output_folderRET_month   = os.path.join(dir_proj, dir_data, "2L1_RET_month")    
output_folderETp         = os.path.join(dir_proj, dir_data, "2L1_ETp_season")    #ETp (= ETc) =Kc*REF
output_folderNPP         = os.path.join(dir_proj, dir_data, "2L2_NPP_season") 

# Make one if the folder does not exit
if not os.path.exists(output_folderT):
    os.makedirs(output_folderT) 
if not os.path.exists(output_folderET):
    os.makedirs(output_folderET) 
if not os.path.exists(output_folderRET):
    os.makedirs(output_folderRET)  
if not os.path.exists(output_folderRET_month):
    os.makedirs(output_folderRET_month)
if not os.path.exists(output_folderETp):
    os.makedirs(output_folderETp) 
if not os.path.exists(output_folderNPP):
    os.makedirs(output_folderNPP) 

## Step 2b - Define function - the function that add rasters between two dates

In [8]:
# summation of raster between two dates
def SumSeason(input_fhs, saveSum, sowing_date, harvesting_date, df_avail):
    """
    Add raster files (input_fhs) between sowing_date and harvesting_date.

    IHE Delft 2019
    Authors: a.chukalla@un-ihe.org
    @author: Abebe Chukalla

    Parameters
    ----------
    input_fhs : raster file
        Files to be added.
    saveSum : folder name
        Folder name where the sum to be saved.
    sowing_date : date in yyyy-mm-dd format
        Starting date of crop growth.
    harvesting_date : date in yyyy-mm-dd format
        Harvesting date of crop.
    df_avail : cube_code of the raster
        Helps to read the date of each raster file.
        
    Returns
    -------
    Sums: array
        Seasonal, sum of the raster files.
    """
    period_dates = pd.date_range(sowing_date, harvesting_date, freq='D')  # generate dates b/n sowing and harvesting dates
    period_fhs   = []

    # collect the rasters if they are within sowing and harvesting date
    for in_fh in input_fhs:
        # get raster id from file name
        raster_id = os.path.split(in_fh)[-1].split('.')[0]               
        # get raster info using raster id
        raster_info = df_avail.loc[df_avail['raster_id']==raster_id]      # the time_code corresponding to raster id
        # get start and end date of raster
        raster_startdate = raster_info['time_code'].iloc[0].split(',')[0] 
        raster_startdate = re.sub(r"[[)]", "", raster_startdate)          
        raster_enddate = raster_info['time_code'].iloc[0].split(',')[-1]  
        raster_enddate = re.sub(r"[[)]", "", raster_enddate)               
        # check if raster belong to period
        if ((raster_startdate in period_dates) or (raster_enddate in period_dates)):
            period_fhs.append(in_fh)

    # add the layers between the sowing and harvesting dates
    period_fhs
    period_fh = period_fhs[0]
    Sums = 0
    for period_fh in period_fhs:
        Sum = gis.OpenAsArray(period_fh, nan_values=True)
        Sums += Sum
           
    # save the array in raster format, name it with the raster_id and sowing and harvesting date
    out_fh = os.path.join(saveSum, raster_id.split('_')[1] + '_' + str(sowing_date) + '_to_' + str(harvesting_date) + '.tif')        
    gis.CreateGeoTiff(out_fh, Sums, driver, NDV, xsize, ysize, GeoT, Projection)  # Save the array 'Sums' as raster
        
    return Sums  

## Step 2c - Calculate seasonal T, ET, RET, ETp, NPP

## i) Calculate seasonal transpiration (T)

In [9]:
# collecting Geoinfo such as projection, the x and y axis
in_fh = input_fhsT[0]  

driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(in_fh)
spatial_extent = (GeoT[0], GeoT[0] + GeoT[1] * xsize, GeoT[3] + GeoT[5] * ysize, GeoT[3])  # get spatial extent of raster

length = len(df_dates)  # length of season

i = 0
while i < length:  
    # calculate the seasonal value and save in output_folderT
    seasonal = SumSeason(input_fhsT, output_folderT, df_dates.SOS[i].date(), df_dates.EOS[i].date(), df_availT)
   
    # calculate the mean, SD
    print ('the mean & SD for ',str(df_dates.SOS[i].date()) + '/' + str(df_dates.EOS[i].date()), '=', np.nanmean(seasonal).round(1),'&',np.nanstd(seasonal).round(1))

    # Plot the raster map
    plt.figure(figsize = (12,8))
    plt.imshow(seasonal, cmap='jet_r', vmin=np.nanmin(seasonal), vmax=np.nanmax(seasonal), extent=spatial_extent)
    plt.colorbar(shrink=0.75, label='T [mm/season]')
    plt.xlabel('Longitude ($^{\circ}$ East)', fontsize=14)  # add axes label
    plt.ylabel('Latitude ($^{\circ}$ North)', fontsize=14)
    plt.title('Transpiration [mm/season] ' + str(df_dates.SOS[i].date()) + '/' + str(df_dates.EOS[i].date()), fontsize=16)
    plt.clim(0,1500)
    plt.show ()
    
    i += 1
    ;

## ii) Calculate seasonal evapotratranspiration

In [10]:
# collecting Geoinfo such as projection, the x and y axis
in_fh = input_fhsET[0]    

driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(in_fh)
spatial_extent = (GeoT[0], GeoT[0] + GeoT[1] * xsize, GeoT[3] + GeoT[5] * ysize, GeoT[3])  # get spatial extent of raster

length = len(df_dates)  # length of seasonn
                
i = 0
while i < length:    
    # calculate the seasonal value and save in output_folder
    seasonal = SumSeason(input_fhsET, output_folderET, df_dates.SOS[i].date(), df_dates.EOS[i].date(), df_availET)
    
    # calculate the mean, SD
    print ('the mean & SD for ',str(df_dates.SOS[i].date()) + '/' + str(df_dates.EOS[i].date()), '=', np.nanmean(seasonal).round(1),'&',np.nanstd(seasonal).round(1))

    # Plot the raster map
    plt.figure(figsize = (12,8))
    plt.imshow(seasonal, cmap='jet_r', vmin=np.nanmin(seasonal), vmax=np.nanmax(seasonal), extent=spatial_extent)
    plt.colorbar(shrink=0.75, label='ETa [mm/season]')
    plt.xlabel('Longitude ($^{\circ}$ East)', fontsize=14)  # add axes label
    plt.ylabel('Latitude ($^{\circ}$ North)', fontsize=14)
    plt.title('Actual evapotranspiration [mm/season] ' + str(df_dates.SOS[i].date()) + '/' + str(df_dates.EOS[i].date()), fontsize=16)
    plt.clim(0,1600)
    plt.show()

    i += 1
    ;

## iii) Calculate seasonal Reference Evapotranspiration(RET)

In [11]:
# collecting Geoinfo such as projection, the x and y axis
in_fh = input_fhsRET[0]
driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(in_fh)
spatial_extent = (GeoT[0], GeoT[0] + GeoT[1] * xsize, GeoT[3] + GeoT[5] * ysize, GeoT[3])  # get spatial extent of raster

length = len(df_dates)  # length of season

i = 0
while i < length:    
    # calculate the seasonal value and save in output_folder
    seasonal = SumSeason(input_fhsRET, output_folderRET, df_dates.SOS[i].date(), df_dates.EOS[i].date(), df_availRET)

    # calculate the mean, SD
    print ('the mean & SD for ',str(df_dates.SOS[i].date()) + '/' + str(df_dates.EOS[i].date()), '=', np.nanmean(seasonal).round(1),'&',np.nanstd(seasonal).round(1))
    
    # Plot the raster map
    plt.figure(figsize = (12,8))
    plt.imshow(seasonal, cmap='jet_r', vmin=np.nanmin(seasonal), vmax=np.nanmax(seasonal), extent=spatial_extent)
    plt.colorbar(shrink=0.75, label='REF [mm/season]')
    plt.xlabel('Longitude ($^{\circ}$ East)', fontsize=14)  # add axes label
    plt.ylabel('Latitude ($^{\circ}$ North)', fontsize=14)
    plt.title('Reference evapotranspiration [mm/season] ' + str(df_dates.SOS[i].date()) + '/' + str(df_dates.EOS[i].date()), fontsize=16)
    plt.show ()

    i += 1
    ;

## iv) Calculate seasonal ETp(ETc) = Kc*RET

In [12]:
# collecting Geoinfo such as projection, the x and y axis
in_fh = input_fhsRET[0]
driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(in_fh)
spatial_extent = (GeoT[0], GeoT[0] + GeoT[1] * xsize, GeoT[3] + GeoT[5] * ysize, GeoT[3])  # get spatial extent of raster

length = len(df_dates)  # length of season

i = 0
while i < length:
    date_s  = df_dates.SOS[i].date()   # the starting date of the season (SOS)
    #ETc_total[i] = 0
    print ('Monthly ETo, ETc in mm/month and Kc[-] in ', date_s.year)
    seasonal = 0
    while date_s < df_dates.EOS[i].date():    # loop in the range of SOS and EOS of the active season 'i' 
        # Increment by a month
        days    = calendar.monthrange(date_s.year,date_s.month)[1] # Number of days in the active month
        date_e  = date_s+datetime.timedelta(days=days-1)           # The end date of the active month
        
        m = date_s.strftime('%B')         # identify the name of the active month
        kc_m =df_kc.loc[df_kc["Months"] == m, 'Kc'].item()  # identify the kc value corresponding to a month
           
        # calculate the monthly value and save in output_folder
        ET0 = SumSeason(input_fhsRET, output_folderRET_month, date_s, date_e, df_availRET) # adding reference ET in the active month
        ETc = kc_m*ET0    # calculate monthly ETP (ETc)

        print (date_s.year, m, np.nanmean(ET0).round(1), np.nanmean(ETc).round(1), kc_m)
        date_s  = date_e+datetime.timedelta(days=1)   # First day of the next month
        seasonal+= ETc
    
    # save the seasonal array in raster format, name it with the raster name, sowing and harvesting date
    out_fh = os.path.join(output_folderETp, 'ETc_' + str(df_dates.SOS[i].date()) + '_to_' + str(df_dates.EOS[i].date()) + '.tif')        
    gis.CreateGeoTiff(out_fh, seasonal, driver, NDV, xsize, ysize, GeoT, Projection)  # Save the array 'Sums' as raster
    print ('The mean seasonal ETc [mm] in ', date_s.year, ' is ', np.nanmean(seasonal).round(1))

    # Plot the raster map
    plt.figure(figsize = (12,8))
    plt.imshow(seasonal, cmap='jet_r', vmin=np.nanmin(seasonal), vmax=np.nanmax(seasonal), extent=spatial_extent)
    plt.colorbar(shrink=0.75, label='ETc=Kc*ETo [mm/season]')
    plt.xlabel('Longitude ($^{\circ}$ East)', fontsize=12)  # add axes label
    plt.ylabel('Latitude ($^{\circ}$ North)', fontsize=12)
    plt.title('ETc ' + str(df_dates.SOS[i].date()) + '/' + str(df_dates.EOS[i].date()), fontsize=12)
    plt.clim()
    plt.show ()

    i += 1
    ;

## v) Calculate seasonal Net Primary Production (NPP)

In [13]:
# collecting Geoinfo such as projection, the x and y axis
in_fh = input_fhsNPP[0]

driver, NDV, xsize, ysize, GeoT, Projection = gis.GetGeoInfo(in_fh)
spatial_extent = (GeoT[0], GeoT[0] + GeoT[1] * xsize, GeoT[3] + GeoT[5] * ysize, GeoT[3])  # get spatial extent of raster

length = len(df_dates)  # length of season

i = 0
while i < length:
    # calculate the seasonal value and save in output_folder
    seasonal = SumSeason(input_fhsNPP, output_folderNPP, df_dates.SOS[i].date(), df_dates.EOS[i].date(), df_availNPP)

    # calculate the mean, SD
    print ('the mean & SD for ',str(df_dates.SOS[i].date()) + '/' + str(df_dates.EOS[i].date()), '=', np.nanmean(seasonal).round(1),'&',np.nanstd(seasonal).round(1))
    
    # Plot the raster map
    plt.figure(figsize = (12,8))
    plt.imshow(seasonal, cmap='jet_r', vmin=np.nanmin(seasonal), vmax=np.nanmax(seasonal), extent=spatial_extent)
    plt.colorbar(shrink=0.75, label='NPP [gC/m2/season]')
    plt.xlabel('Longitude ($^{\circ}$ East)', fontsize=14)  # add axes label
    plt.ylabel('Latitude ($^{\circ}$ North)', fontsize=14)
    plt.title('Net primary production [gC/m2/season] ' + str(df_dates.SOS[i].date()) + '/' + str(df_dates.EOS[i].date()), fontsize=16)
    plt.show ()
    
    i += 1 
    ;